In [1]:
import pandas as pd
import numpy as np
seed = np.random.seed(42)
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style="ticks", color_codes=True)

pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',50)

%matplotlib inline
best_model = None
best_score = 0
hospital_name = 'host3'

In [2]:
data = pd.read_csv('../data/hospital' + str(3) + '_data.csv')

In [3]:
from sklearn.preprocessing import Imputer,StandardScaler,LabelEncoder
from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute, MICE

/Users/refikcanmalli/anaconda3/envs/vodafone/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
data_e = data[data['IPG'] != 1]
data_e = data_e.drop('IPG', axis = 1);
data_e = data_e.drop('sbjBeatConsidered', axis = 1);
data_e = data_e.drop('numRRaveraged', axis = 1);
data_e = data_e.drop('Soggetti', axis = 1);
# df_columns = data.columns.tolist()
# matching_columns1 = [s for s in df_columns if "Parea" in s]
# matching_columns2 = [s for s in df_columns if "Pdur" in s]
# mc = matching_columns1 + matching_columns2
# data_e = data_e.dropna(subset=mc)
data_e['PCneg'].fillna(0, inplace=True)
data_e['AFclass'].replace('persistierend (>7 Tage, EKV)', 1, inplace=True)
data_e['AFclass'].replace('paroxysmal', 0, inplace=True)
categorical_variables_mask = data_e.dtypes==object
categorical_variables = data_e.columns[categorical_variables_mask]
for v in categorical_variables:
    data_e[v] = LabelEncoder().fit_transform(data_e[v])
del categorical_variables_mask,categorical_variables
data_e = pd.DataFrame(data=MICE(n_imputations=100, impute_type='col', n_nearest_columns=5).complete(data_e.as_matrix()), columns=data_e.columns, index=data_e.index);
y_df = data_e['AFclass'].copy()
x_df = data_e.drop('AFclass',axis=1, inplace=False)

[MICE] Completing matrix with shape (274, 69)
[MICE] Starting imputation round 1/110, elapsed time 0.014
[MICE] Starting imputation round 2/110, elapsed time 0.098
[MICE] Starting imputation round 3/110, elapsed time 0.194
[MICE] Starting imputation round 4/110, elapsed time 0.327
[MICE] Starting imputation round 5/110, elapsed time 0.396
[MICE] Starting imputation round 6/110, elapsed time 0.483
[MICE] Starting imputation round 7/110, elapsed time 0.597
[MICE] Starting imputation round 8/110, elapsed time 0.675
[MICE] Starting imputation round 9/110, elapsed time 0.757
[MICE] Starting imputation round 10/110, elapsed time 0.856
[MICE] Starting imputation round 11/110, elapsed time 0.934
[MICE] Starting imputation round 12/110, elapsed time 1.019
[MICE] Starting imputation round 13/110, elapsed time 1.083
[MICE] Starting imputation round 14/110, elapsed time 1.222
[MICE] Starting imputation round 15/110, elapsed time 1.315
[MICE] Starting imputation round 16/110, elapsed time 1.398
[MI

In [5]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,ExtraTreesClassifier,AdaBoostClassifier
#from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

#f1_scorer = make_scorer(f1_score, average='macro',labels=Y.unique().tolist())
#myscorer = f1_scorer
myscorer = 'accuracy'

In [6]:
balancer = SMOTE(random_state=42)

In [7]:
def show_f_importance(estimator,train_data):
    importances = estimator.feature_importances_
    indices = np.argsort(importances)[::-1]
    # Print the feature ranking
    print("Feature ranking:")

    for f in range(train_data.shape[1]):
        print("%d. feature %d - %s (%f)" % (f + 1, indices[f],train_data.columns[indices[f]], importances[indices[f]]))


In [8]:
param_grid = {"clf__strategy": ['most_frequent','stratified','prior','uniform']}

dummy = DummyClassifier(random_state=42)
plist = []
plist.append(('balancer', balancer))
plist.append(('clf', dummy))
pipeline = Pipeline(plist)

gs_dummy = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1, scoring=myscorer, n_jobs=-1)

In [9]:
gs_dummy.fit(x_df, y_df);

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.2s finished


In [10]:
gs_dummy.best_params_

{'clf__strategy': 'most_frequent'}

In [11]:
gs_dummy.best_score_

0.6824817518248175

In [12]:
param_grid = {"clf__max_depth": [10,30,50],
              "clf__criterion": ['gini', 'entropy'],
              "clf__n_estimators": [100,200,500,1000],
              "clf__random_state": [42],
              "clf__class_weight" : [None, 'balanced']}
# param_grid = {"classifier__max_depth": [30],
#               "classifier__n_estimators": [1000]}


rf = RandomForestClassifier()

plist = []
plist.append(('balancer', balancer))
plist.append(('clf', rf))
pipeline = Pipeline(plist)
gs_rf = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1, scoring=myscorer, n_jobs=-1)

In [13]:
gs_rf.fit(x_df, y_df);

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   51.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  6.1min finished


In [14]:
gs_rf.best_params_

{'clf__class_weight': None,
 'clf__criterion': 'gini',
 'clf__max_depth': 10,
 'clf__n_estimators': 500,
 'clf__random_state': 42}

In [15]:
gs_rf.best_score_


0.6313868613138686

In [16]:
if gs_rf.best_score_ > best_score:
    best_model = gs_rf
    best_score = gs_rf.best_score_

In [17]:
show_f_importance(gs_rf.best_estimator_.named_steps['clf'],x_df)

Feature ranking:
1. feature 66 - patsex (0.040571)
2. feature 48 - Parea_ECG_aVL (0.030863)
3. feature 4 - QRS (0.029480)
4. feature 5 - PQ (0.027213)
5. feature 55 - Pfwhm_ECG_V3 (0.027117)
6. feature 39 - Parea_ECG_V3 (0.025027)
7. feature 3 - QTn (0.024097)
8. feature 50 - Pfwhm_ECG_I (0.022645)
9. feature 62 - Pfwhm_ECG_V9 (0.021918)
10. feature 36 - Parea_ECG_III (0.021754)
11. feature 63 - Pfwhm_ECG_aVF (0.020380)
12. feature 40 - Parea_ECG_V3R (0.018565)
13. feature 56 - Pfwhm_ECG_V3R (0.018163)
14. feature 6 - PCpos (0.018089)
15. feature 57 - Pfwhm_ECG_V4 (0.017235)
16. feature 41 - Parea_ECG_V4 (0.017031)
17. feature 44 - Parea_ECG_V6 (0.016763)
18. feature 45 - Parea_ECG_V8 (0.016672)
19. feature 2 - QR (0.016559)
20. feature 54 - Pfwhm_ECG_V2 (0.016495)
21. feature 17 - Pdur16iqr (0.016399)
22. feature 42 - Parea_ECG_V4R (0.016058)
23. feature 34 - Parea_ECG_I (0.016053)
24. feature 16 - Pdur16std (0.016041)
25. feature 43 - Parea_ECG_V5 (0.015978)
26. feature 46 - Parea_EC

In [18]:
param_grid = {"clf__max_depth": [3,5,8],
              "clf__n_estimators": [100,200,500,1000],
              "clf__min_samples_leaf": [1,3,5,10]}
# param_grid = {"classifier__n_estimators": [50],
#               "classifier__max_depth": [5],
# 

gb = GradientBoostingClassifier(random_state=42)
plist = []
plist.append(('balancer', balancer))
plist.append(('clf', gb))
pipeline = Pipeline(plist)
gs_gb = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1,  scoring=myscorer, n_jobs=-1)

In [19]:
gs_gb.fit(x_df, y_df);

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   47.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  4.3min finished


In [20]:
gs_gb.best_params_

{'clf__max_depth': 8, 'clf__min_samples_leaf': 5, 'clf__n_estimators': 200}

In [21]:
gs_gb.best_score_

0.635036496350365

In [22]:
if gs_gb.best_score_ > best_score:
    best_model = gs_gb
    best_score = gs_gb.best_score_

In [23]:
show_f_importance(gs_gb.best_estimator_.named_steps['clf'],x_df)

Feature ranking:
1. feature 48 - Parea_ECG_aVL (0.025212)
2. feature 66 - patsex (0.023294)
3. feature 5 - PQ (0.023003)
4. feature 55 - Pfwhm_ECG_V3 (0.020932)
5. feature 50 - Pfwhm_ECG_I (0.020516)
6. feature 3 - QTn (0.019531)
7. feature 39 - Parea_ECG_V3 (0.017993)
8. feature 4 - QRS (0.017889)
9. feature 64 - Pfwhm_ECG_aVL (0.016608)
10. feature 54 - Pfwhm_ECG_V2 (0.014943)
11. feature 17 - Pdur16iqr (0.014734)
12. feature 41 - Parea_ECG_V4 (0.014088)
13. feature 37 - Parea_ECG_V1 (0.013891)
14. feature 56 - Pfwhm_ECG_V3R (0.012591)
15. feature 63 - Pfwhm_ECG_aVF (0.012512)
16. feature 38 - Parea_ECG_V2 (0.012348)
17. feature 2 - QR (0.012146)
18. feature 40 - Parea_ECG_V3R (0.011563)
19. feature 45 - Parea_ECG_V8 (0.010967)
20. feature 62 - Pfwhm_ECG_V9 (0.010739)
21. feature 36 - Parea_ECG_III (0.010439)
22. feature 67 - Age (0.010358)
23. feature 57 - Pfwhm_ECG_V4 (0.009840)
24. feature 0 - RR (0.009526)
25. feature 46 - Parea_ECG_V9 (0.009318)
26. feature 53 - Pfwhm_ECG_V1 (0.

In [24]:
param_grid = {"clf__learning_rate": [0.1,0.3,0.7,0.8],
              "clf__n_estimators": [100,1000,2000,2500,3000,5000]}

ab = AdaBoostClassifier(random_state=42)

plist = []
plist.append(('balancer', balancer))
plist.append(('clf', ab))
pipeline = Pipeline(plist)

gs_ab = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1, scoring=myscorer, n_jobs=-1)

In [25]:
gs_ab.fit(x_df, y_df);

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed: 21.5min finished


In [26]:
gs_ab.best_params_

{'clf__learning_rate': 0.1, 'clf__n_estimators': 1000}

In [27]:
gs_ab.best_score_

0.5875912408759124

In [28]:

if gs_ab.best_score_ > best_score:
    best_model = gs_ab
    best_score = gs_ab.best_score_

In [29]:
show_f_importance(gs_ab.best_estimator_.named_steps['clf'],x_df)

Feature ranking:
1. feature 5 - PQ (0.072000)
2. feature 63 - Pfwhm_ECG_aVF (0.060000)
3. feature 44 - Parea_ECG_V6 (0.056000)
4. feature 18 - Pdur_ECG_I (0.056000)
5. feature 50 - Pfwhm_ECG_I (0.050000)
6. feature 38 - Parea_ECG_V2 (0.039000)
7. feature 48 - Parea_ECG_aVL (0.038000)
8. feature 3 - QTn (0.033000)
9. feature 54 - Pfwhm_ECG_V2 (0.033000)
10. feature 4 - QRS (0.032000)
11. feature 55 - Pfwhm_ECG_V3 (0.032000)
12. feature 39 - Parea_ECG_V3 (0.029000)
13. feature 66 - patsex (0.027000)
14. feature 37 - Parea_ECG_V1 (0.026000)
15. feature 62 - Pfwhm_ECG_V9 (0.026000)
16. feature 23 - Pdur_ECG_V3 (0.024000)
17. feature 45 - Parea_ECG_V8 (0.022000)
18. feature 36 - Parea_ECG_III (0.020000)
19. feature 1 - QTm_old (0.019000)
20. feature 22 - Pdur_ECG_V2 (0.019000)
21. feature 64 - Pfwhm_ECG_aVL (0.019000)
22. feature 51 - Pfwhm_ECG_II (0.018000)
23. feature 16 - Pdur16std (0.018000)
24. feature 25 - Pdur_ECG_V4 (0.017000)
25. feature 46 - Parea_ECG_V9 (0.017000)
26. feature 29 

In [30]:
param_grid = {"clf__max_depth": [3,5,8,10,15,30],
              "clf__n_estimators": [100,200,500,1000,1500],
              "clf__min_samples_leaf": [1,3,5,10]}

et = ExtraTreesClassifier(random_state=42)

plist = []
plist.append(('balancer', balancer))
plist.append(('clf', et))
pipeline = Pipeline(plist)

gs_et = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1,  scoring=myscorer, n_jobs=-1)

In [31]:
gs_et.fit(x_df, y_df);

Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   48.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed: 11.0min finished


In [32]:
gs_et.best_params_

{'clf__max_depth': 30, 'clf__min_samples_leaf': 1, 'clf__n_estimators': 100}

In [33]:
gs_et.best_score_

0.656934306569343

In [34]:
if gs_et.best_score_ > best_score:
    best_model = gs_et
    best_score = gs_et.best_score_

In [35]:
show_f_importance(gs_et.best_estimator_.named_steps['clf'],x_df)

Feature ranking:
1. feature 66 - patsex (0.053928)
2. feature 5 - PQ (0.021919)
3. feature 50 - Pfwhm_ECG_I (0.021834)
4. feature 4 - QRS (0.019984)
5. feature 40 - Parea_ECG_V3R (0.019242)
6. feature 48 - Parea_ECG_aVL (0.018773)
7. feature 6 - PCpos (0.018446)
8. feature 55 - Pfwhm_ECG_V3 (0.017925)
9. feature 47 - Parea_ECG_aVF (0.017737)
10. feature 57 - Pfwhm_ECG_V4 (0.017678)
11. feature 3 - QTn (0.017459)
12. feature 54 - Pfwhm_ECG_V2 (0.017098)
13. feature 39 - Parea_ECG_V3 (0.016816)
14. feature 62 - Pfwhm_ECG_V9 (0.016613)
15. feature 43 - Parea_ECG_V5 (0.016586)
16. feature 34 - Parea_ECG_I (0.016546)
17. feature 36 - Parea_ECG_III (0.016414)
18. feature 16 - Pdur16std (0.016135)
19. feature 56 - Pfwhm_ECG_V3R (0.016042)
20. feature 63 - Pfwhm_ECG_aVF (0.015928)
21. feature 41 - Parea_ECG_V4 (0.015902)
22. feature 2 - QR (0.015419)
23. feature 42 - Parea_ECG_V4R (0.015170)
24. feature 64 - Pfwhm_ECG_aVL (0.015079)
25. feature 0 - RR (0.014912)
26. feature 17 - Pdur16iqr (0.0

In [36]:
cdata = pd.read_csv('../data/central' +'_data.csv')
cdata.shape

(275, 73)

In [37]:
data_e = cdata[cdata['IPG'] != 1]
data_e = data_e.drop('IPG', axis = 1);
data_e = data_e.drop('sbjBeatConsidered', axis = 1);
data_e = data_e.drop('numRRaveraged', axis = 1);
data_e = data_e.drop('Soggetti', axis = 1);
# df_columns = data.columns.tolist()
# matching_columns1 = [s for s in df_columns if "Parea" in s]
# matching_columns2 = [s for s in df_columns if "Pdur" in s]
# mc = matching_columns1 + matching_columns2
# data_e = data_e.dropna(subset=mc)
data_e['PCneg'].fillna(0, inplace=True)
data_e['AFclass'].replace('persistierend (>7 Tage, EKV)', 1, inplace=True)
data_e['AFclass'].replace('paroxysmal', 0, inplace=True)
categorical_variables_mask = data_e.dtypes==object
categorical_variables = data_e.columns[categorical_variables_mask]
for v in categorical_variables:
    data_e[v] = LabelEncoder().fit_transform(data_e[v])
del categorical_variables_mask,categorical_variables
data_e = pd.DataFrame(data=MICE(n_imputations=100, impute_type='col', n_nearest_columns=5).complete(data_e.as_matrix()), columns=data_e.columns, index=data_e.index);
cy_df = data_e['AFclass'].copy()
cx_df = data_e.drop('AFclass',axis=1, inplace=False)

[MICE] Completing matrix with shape (270, 69)
[MICE] Starting imputation round 1/110, elapsed time 0.003
[MICE] Starting imputation round 2/110, elapsed time 0.031
[MICE] Starting imputation round 3/110, elapsed time 0.079
[MICE] Starting imputation round 4/110, elapsed time 0.181
[MICE] Starting imputation round 5/110, elapsed time 0.258
[MICE] Starting imputation round 6/110, elapsed time 0.354
[MICE] Starting imputation round 7/110, elapsed time 0.448
[MICE] Starting imputation round 8/110, elapsed time 0.560
[MICE] Starting imputation round 9/110, elapsed time 0.671
[MICE] Starting imputation round 10/110, elapsed time 0.739
[MICE] Starting imputation round 11/110, elapsed time 0.763
[MICE] Starting imputation round 12/110, elapsed time 0.787
[MICE] Starting imputation round 13/110, elapsed time 0.812
[MICE] Starting imputation round 14/110, elapsed time 0.853
[MICE] Starting imputation round 15/110, elapsed time 0.878
[MICE] Starting imputation round 16/110, elapsed time 0.902
[MI

In [38]:
from sklearn.metrics import accuracy_score
p = best_model.predict(cx_df);
scores = pd.DataFrame({hospital_name: p})
scores.to_csv('../data/' + hospital_name +'_predictions.csv', index=False)
np.save('../data/' + hospital_name +'_acc',best_score)
accuracy_score(cy_df, p)

0.6481481481481481

In [39]:
best_score

0.656934306569343